# 01 - Initialisation et Collecte des Données

## Objectif
Ce notebook permet d'initialiser l'environnement Big Data avec:
- Configuration de Spark avec Delta Lake
- Configuration de Kafka pour la collecte en temps réel
- Collecte des données depuis une API
- Stockage dans le Data Lake (architecture Médaillon)

---

## 1. Installation et Import des Librairies

### Installation des dépendances

In [ ]:
# Installation des dépendances depuis requirements.txt

# !pip install -r requirements.txt

# Note: Si vous utilisez Docker, les dépendances sont déjà installées dans l'image

### Import des librairies

In [ ]:
# Spark et Delta Lake
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta import *

# Kafka
from kafka import KafkaProducer, KafkaConsumer
from kafka.admin import KafkaAdminClient, NewTopic

# Traitement et visualisation
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Utilitaires
import json
import requests
import time
from datetime import datetime
import os
from dotenv import load_dotenv

load_dotenv()

print("Toutes les librairies ont été importées avec succès")

✓ Toutes les librairies ont été importées avec succès


---
## 2. Configuration de SparkSession avec Delta Lake

In [ ]:
builder = SparkSession.builder \
    .appName("BigData-DataLake-Medallion") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

spark.sparkContext.setLogLevel("WARN")

print(f"✓ SparkSession créée avec succès")
print(f"  Version Spark: {spark.version}")
print(f"  Application ID: {spark.sparkContext.applicationId}")

:: loading settings :: url = jar:file:/opt/conda/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9d68febb-37af-4cd4-be03-8db7d3264b81;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.2 in central
	found io.delta#delta-storage;3.3.2 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-spark_2.12/3.3.2/delta-spark_2.12-3.3.2.jar ...
	[SUCCESSFUL ] io.delta#delta-spark_2.12;3.3.2!delta-spark_2.12.jar (494ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/3.3.2/delta-storage-3.3.2.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;3.3.2!delta-storage.jar (65ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.9.3/antlr4-runtime-4.9.3.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.9.3!antlr4-runtime.jar (84ms)
:: resolution report :: resolve 1340ms :: a

✓ SparkSession créée avec succès
  Version Spark: 3.5.3
  Application ID: local-1768566906634


---
## 3. Architecture Médaillon

### Principe de l'architecture en couches

Organisation des données en trois couches:

#### 🥉 **Bronze** - Données brutes
- **Format**: Delta Lake (avec support Parquet)
- **Contenu**: Données brutes exactement comme reçues de la source
- **Objectif**: Conservation historique complète, pas de transformation

#### 🥈 **Silver** - Données nettoyées
- **Format**: Delta Lake
- **Contenu**: Données validées, dédupliquées, enrichies
- **Objectif**: Données de qualité pour l'analyse

#### 🥇 **Gold** - Données agrégées
- **Format**: Delta Lake
- **Contenu**: Données agrégées pour cas d'usage spécifiques
- **Objectif**: Optimisation pour les dashboard, rapports, ...

In [ ]:
DATALAKE_PATH = "./datalake"

BRONZE_PATH = f"{DATALAKE_PATH}/bronze"
SILVER_PATH = f"{DATALAKE_PATH}/silver"
GOLD_PATH = f"{DATALAKE_PATH}/gold"

os.makedirs(BRONZE_PATH, exist_ok=True)
os.makedirs(SILVER_PATH, exist_ok=True)
os.makedirs(GOLD_PATH, exist_ok=True)

print("Structure du Data Lake créée:")
print(f"Bronze: {BRONZE_PATH}")
print(f"Silver: {SILVER_PATH}")
print(f"Gold: {GOLD_PATH}")

✓ Structure du Data Lake créée:
  Bronze: ./datalake/bronze
  Silver: ./datalake/silver
  Gold: ./datalake/gold


---
## 4. Configuration de Kafka

### Architecture de streaming
On utilise ici Kafka pour la collecte en temps réel des données depuis l'API.

In [ ]:
KAFKA_BOOTSTRAP_SERVERS = os.getenv('KAFKA_BOOTSTRAP_SERVERS', 'kafka1:9092')
KAFKA_TOPIC = os.getenv('KAFKA_TOPIC', 'opensky-data')

print(f"Configuration Kafka:")
print(f"  Bootstrap servers: {KAFKA_BOOTSTRAP_SERVERS}")
print(f"  Topic: {KAFKA_TOPIC}")

Configuration Kafka:
  Bootstrap servers: kafka1:9092
  Topic: opensky-data


### Création du topic Kafka

Le topic Kafka permet de stocker un flux de données de manière ordonnée et immuable. Nous utilisons trois partitions afin de permettre la parallélisation.

In [ ]:
def create_kafka_topic(topic_name, num_partitions=3, replication_factor=1):
    """
    Crée un topic Kafka s'il n'existe pas déjà.
    
    Args:
        topic_name: Nom du topic à créer
        num_partitions: Nombre de partitions (par défaut 3)
        replication_factor: Facteur de réplication (par défaut 1)
    """
    try:
        admin_client = KafkaAdminClient(
            bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
            client_id='admin'
        )
        
        existing_topics = admin_client.list_topics()
        
        if topic_name not in existing_topics:
            topic = NewTopic(
                name=topic_name,
                num_partitions=num_partitions,
                replication_factor=replication_factor
            )
            admin_client.create_topics(new_topics=[topic], validate_only=False)
            print(f"✓ Topic '{topic_name}' créé avec succès")
        else:
            print(f"ℹ Topic '{topic_name}' existe déjà")
            
        admin_client.close()
        
    except Exception as e:
        print(f"Erreur lors de la création du topic: {e}")

create_kafka_topic(KAFKA_TOPIC)

✓ Topic 'opensky-data' créé avec succès


### Configuration du producteur Kafka

In [ ]:
def get_kafka_producer():
    """
    Crée et retourne un producteur Kafka configuré.
    
    Returns:
        KafkaProducer: Producteur Kafka configuré avec sérialisation JSON
    """
    try:
        producer = KafkaProducer(
            bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
            value_serializer=lambda v: json.dumps(v).encode('utf-8'),
            key_serializer=lambda k: k.encode('utf-8') if k else None,
            acks='all',
            retries=3,
            max_in_flight_requests_per_connection=1
        )
        print("Producteur Kafka créé avec succès")
        return producer
    except Exception as e:
        print(f"Erreur lors de la création du producteur: {e}")
        return None

producer = get_kafka_producer()

✓ Producteur Kafka créé avec succès
